In [1]:
import os
from pathlib import Path
# Change cwd to the project root (parent of 'notebooks/')
os.chdir(Path.cwd().parent)
Path.cwd()

PosixPath('/Users/jbrandt/code/birddog')

In [2]:
#import notebook_utils
from time import sleep
import json
from birddog.translate import (
    is_english, 
    translation,
    queue_translation,
    cancel_translation,
    is_translation_running,
    GoogleCloudTranslator)
from birddog.core import Archive

2025-05-11 11:30:15,579 [INFO] Using Google Cloud translation API (credentials file:/Users/jbrandt/code/birddog/google-cloud-translate-key.json)
2025-05-11 11:30:15,595 [INFO] Using local folder /Users/jbrandt/code/birddog/.cache for storage.


In [3]:
text = 'Перейменування: 1941-1945 Рейхскомісаріат України, м. Рівне.'

In [4]:
translation(text)

'Renaming: 1941-1945 Reich Commissariat of Ukraine, Rivne.'

In [5]:
is_english(text)

False

In [6]:
is_english(translation(text))

True

In [7]:
translation([text, text, text, text])

['Renaming: 1941-1945 Reich Commissariat of Ukraine, Rivne.',
 'Renaming: 1941-1945 Reich Commissariat of Ukraine, Rivne.',
 'Renaming: 1941-1945 Reich Commissariat of Ukraine, Rivne.',
 'Renaming: 1941-1945 Reich Commissariat of Ukraine, Rivne.']

In [8]:
gen_batch = False
file_name = 'test/resources/dazho_d_descriptions.json'
if gen_batch:
    archive = Archive('DAZHO', 'D')
    translation_batch = [child[1]['text']['uk'] for child in archive.children]
    print('saving', len(translation_batch), 'items.')
    with open(file_name, 'w', encoding="utf8") as file:
        file.write(json.dumps(translation_batch))
else:
    with open(file_name, 'r', encoding="utf8") as file:
        translation_batch = json.loads(file.read())
        print('loading', len(translation_batch), 'items.')

loading 475 items.


In [9]:
def progress_cb(task_id, progress, total):
    print(f'progress[{task_id}]: {progress}/{total} {float(progress)/float(total)*100.:.1f}%')

In [10]:
def completion_cb(task_id, results):
    print(f'completed[{task_id}]:')
    for item in results[:10]:
        print('    ', item)

In [11]:
task_id = queue_translation(translation_batch[:80], progress_cb, completion_cb)
print('Queued', task_id)

Queued 4e61429d-1dc3-41a7-9d00-e6684bb89b60


In [12]:
tasks = [
    queue_translation(translation_batch[batch[0]:batch[1]], progress_cb, completion_cb)
    for batch in ((0, 100), (100, 150), (150, 350), (350, 500)) ]
print('Queued:', tasks)

Queued: ['b5171b3d-073d-43f3-8814-ebb0f4b5dc88', 'ca45bd28-0fac-46f6-ac6d-62c8bd82b859', 'cb172175-7333-4135-b0fd-6f7adb940221', '3b4ff4d5-5ee1-4fba-b298-be441c632cac']


In [13]:
#sleep(10)
cancel_translation(tasks[3])

False

In [14]:
is_translation_running(tasks[3])

False

In [15]:
is_translation_running()

False

In [16]:
is_translation_running()

False

In [3]:
sample = [
    "Р-1",
    "Р-2",
    "Р-3",
    "Р-4",
    "Р-5",
    "Р-6",
    "Р-7",
    "Р-10"
    ]

In [4]:
translation(sample)

['P-1', 'R-2', 'R-3', 'R-4', 'P-5', 'R-6', 'R-7', 'P-10']

In [9]:
sample2 = [text[:3] for text in sample]
sample2

['Р-1', 'Р-2', 'Р-3', 'Р-4', 'Р-5', 'Р-6', 'Р-7', 'Р-1']

In [10]:
translation(sample2)

['P-1', 'R-2', 'R-3', 'R-4', 'P-5', 'R-6', 'R-7', 'P-1']

In [11]:
cyP = sample[0][:1]
cyP

'Р'

In [19]:
sample3 = [f'{cyP}-{i}' for i in range(100)]

In [20]:
trans3 = translation(sample3)
for trans in trans3:
    if trans[0] == 'P':
        print(trans)

P-0
P-1
P-5
P-8
P-10
